In [1]:
from preprocessing import preprocessing_data
import os
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [51]:
# Original data

retards_df = pd.read_csv("data/regularite-mensuelle-tgv-aqst.csv", sep=';')
#retards_df.iloc[1]
retards_df["date"]


0       2018-01
1       2018-01
2       2018-01
3       2018-01
4       2018-01
         ...   
8149    2023-06
8150    2023-06
8151    2023-06
8152    2023-06
8153    2023-06
Name: date, Length: 8154, dtype: object

In [42]:
retards_df.columns


Index(['date', 'service', 'gare_depart', 'gare_arrivee', 'duree_moyenne',
       'nb_train_prevu', 'nb_annulation', 'commentaire_annulation',
       'nb_train_depart_retard', 'retard_moyen_depart',
       'retard_moyen_tous_trains_depart', 'commentaire_retards_depart',
       'nb_train_retard_arrivee', 'retard_moyen_arrivee',
       'retard_moyen_tous_trains_arrivee', 'commentaires_retard_arrivee',
       'nb_train_retard_sup_15', 'retard_moyen_trains_retard_sup15',
       'nb_train_retard_sup_30', 'nb_train_retard_sup_60',
       'prct_cause_externe', 'prct_cause_infra', 'prct_cause_gestion_trafic',
       'prct_cause_materiel_roulant', 'prct_cause_gestion_gare',
       'prct_cause_prise_en_charge_voyageurs'],
      dtype='object')

In [2]:
# Preprocess data

train_df_X, train_df_Y, test_df_X, test_df_Y  = preprocessing_data("data/regularite-mensuelle-tgv-aqst.csv")


In [7]:
# Import random forest

from sklearn.ensemble import RandomForestRegressor


In [8]:
# Random forest

rf = RandomForestRegressor(n_estimators = 1000, random_state =42)

rf.fit(train_df_X,train_df_Y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [9]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_df_X)
# Calculate the absolute errors
errors = abs(predictions - test_df_Y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


Mean Absolute Error: prct_cause_externe                      0.71
prct_cause_infra                        0.66
prct_cause_gestion_trafic               0.63
prct_cause_materiel_roulant             0.65
prct_cause_gestion_gare                 0.64
prct_cause_prise_en_charge_voyageurs    0.60
total_retard_mean                       0.49
total_retard_alltrains_mean             0.27
dtype: float64 degrees.


c:\Users\diego\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [10]:
df_retard_mean = train_df_X.copy()

df_retard_mean["total_retard_mean"] = train_df_Y["total_retard_mean"]

In [37]:
train_df_Y.head()


,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs,total_retard_mean,total_retard_alltrains_mean
0,-0.321926,-0.256907,1.353722,-0.580885,-0.830977,0.698461,-0.020550,1.291139
1,-0.153366,0.143358,0.801676,0.360752,-0.830977,-0.695428,0.617756,0.281902
2,-0.131766,-0.724090,-0.570417,1.251919,1.774462,-0.695428,-1.085464,-0.562453
3,-0.087596,-0.251071,-0.331149,0.016526,0.121010,1.180960,0.147451,0.426718
4,-0.573101,0.343134,1.462334,-0.694761,0.294099,-0.695428,-0.664099,-0.346511


In [3]:
from pycaret.regression import *
exp_reg = setup(data=train_df_X.join(train_df_Y["total_retard_mean"]), target='total_retard_mean', session_id=42)

best_model_target1 = compare_models()

# Tune the best model for target1
tuned_model_target1 = tune_model(best_model_target1)

# Predict target1

predictions_target1 = predict_model(tuned_model_target1, data=test_df_X)

,Description,Value
0,Session id,42
1,Target,total_retard_mean
2,Target type,Regression
3,Original data shape,"(5942, 125)"
4,Transformed data shape,"(5942, 125)"
5,Transformed train set shape,"(4159, 125)"
6,Transformed test set shape,"(1783, 125)"
7,Numeric features,124
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.5100,0.6116,0.7801,0.3550,0.3286,3.4021,0.4810
lightgbm,Light Gradient Boosting Machine,0.5082,0.6184,0.7842,0.3527,0.3283,3.1505,0.1240
gbr,Gradient Boosting Regressor,0.5137,0.6166,0.7833,0.3509,0.3387,2.8793,0.2210
xgboost,Extreme Gradient Boosting,0.5223,0.6736,0.8182,0.2845,0.3362,3.5166,0.5340
et,Extra Trees Regressor,0.5347,0.6772,0.8217,0.2799,0.3349,3.5616,0.4930
br,Bayesian Ridge,0.5494,0.7410,0.8559,0.2342,0.3598,2.8869,0.0290
ridge,Ridge Regression,0.5496,0.7394,0.8556,0.2337,0.3582,3.1194,0.0150
knn,K Neighbors Regressor,0.5563,0.7322,0.8541,0.2277,0.3437,3.6956,0.0290
huber,Huber Regressor,0.5350,0.7700,0.8702,0.2107,0.3571,2.9305,0.2280
omp,Orthogonal Matching Pursuit,0.5839,0.7925,0.8860,0.1778,0.3713,2.9465,0.0150


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.5799,0.8226,0.9069,0.3004,0.3742,1.8796
1,0.5539,0.6601,0.8125,0.3475,0.3204,2.6990
2,0.5944,0.6903,0.8308,0.3694,0.3393,4.8695
3,0.5138,0.6595,0.8121,0.2959,0.3178,2.0381
4,0.5256,0.4905,0.7004,0.3457,0.3244,1.7680
5,0.5726,0.6904,0.8309,0.2854,0.3733,2.0244
6,0.5360,0.7036,0.8388,0.3355,0.3283,4.1052
7,0.5794,0.7810,0.8838,0.3380,0.3490,4.2830
8,0.5277,0.5412,0.7357,0.1778,0.3224,6.7669


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [34]:
pd.concat([train_df_X, train_df_Y["total_retard_alltrains_mean"]], axis=0, ignore_index=True)

,duree_moyenne,nb_train_prevu,nb_train_depart_retard,nb_train_retard_arrivee,gare_depart_AIX EN PROVENCE TGV,gare_depart_ANGERS SAINT LAUD,gare_depart_ANGOULEME,gare_depart_ANNECY,gare_depart_ARRAS,gare_depart_AVIGNON TGV,...,gare_arrivee_TOULON,gare_arrivee_TOULOUSE MATABIAU,gare_arrivee_TOURCOING,gare_arrivee_TOURS,gare_arrivee_VALENCE ALIXAN TGV,gare_arrivee_VANNES,gare_arrivee_ZURICH,service_International,service_National,0
0,0.699656,-0.795134,0.271807,-0.219193,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,0.642438,-0.352109,-0.646545,-0.084638,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
2,-0.971104,-0.001141,1.710190,-0.521943,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
3,-0.753676,1.086284,1.356127,1.462750,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
4,-0.078506,-0.162241,-0.192901,-0.286471,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.680882
11880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.210568
11881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276919
11882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.319663


In [35]:
train_df_Y["total_retard_alltrains_mean"]

0       1.291139
1       0.281902
2      -0.562453
3       0.426718
4      -0.346511
          ...   
5937    0.680882
5938    2.210568
5939    0.276919
5940   -0.319663
5941    0.499544
Name: total_retard_alltrains_mean, Length: 5942, dtype: float64

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:19:41
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5046,0.6659,0.8030,0.3218,0.3262,3.9078,0.1050
gbr,Gradient Boosting Regressor,0.5175,0.6945,0.8200,0.2934,0.3423,3.3300,0.1710
rf,Random Forest Regressor,0.5131,0.6966,0.8223,0.2882,0.3290,3.8492,0.3880
et,Extra Trees Regressor,0.5286,0.7155,0.8368,0.2566,0.3287,4.1157,0.3340
xgboost,Extreme Gradient Boosting,0.5262,0.7403,0.8497,0.2338,0.3359,3.9605,0.6920
lr,Linear Regression,0.5446,0.7603,0.8607,0.2187,0.3511,2.8707,0.3590
ridge,Ridge Regression,0.5431,0.7618,0.8614,0.2175,0.3522,2.8654,0.0100
br,Bayesian Ridge,0.5437,0.7637,0.8623,0.2161,0.3560,2.6810,0.0220
huber,Huber Regressor,0.5263,0.7742,0.8664,0.2106,0.3522,2.8080,0.1810
knn,K Neighbors Regressor,0.5503,0.7845,0.8745,0.1903,0.3390,3.5448,0.0230


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4919,0.4236,0.6509,0.3273,0.3167,2.1743
1,0.5559,0.7701,0.8776,0.3399,0.3715,4.0813
2,0.4839,0.5288,0.7272,0.3310,0.3217,2.6575
3,0.5521,1.1949,1.0931,0.1752,0.3612,1.5435
4,0.5095,0.6507,0.8066,0.3020,0.3330,4.7595
5,0.5598,1.0765,1.0376,0.2053,0.3605,2.6669
6,0.5057,0.5188,0.7203,0.3518,0.3301,2.2777
7,0.5099,0.4424,0.6652,0.3537,0.3040,3.4216
8,0.5425,0.8165,0.9036,0.2042,0.3381,9.7196


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [21]:
# Blending 

top_models = compare_models(n_select=3)  # You can select any number of top models based on your preference

# Blend the top models
blender = blend_models(estimator_list=top_models)

# Evaluate the blended model
evaluate_model(blender)



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5046,0.6659,0.8030,0.3218,0.3262,3.9078,0.0980
gbr,Gradient Boosting Regressor,0.5175,0.6945,0.8200,0.2934,0.3423,3.3300,0.1590
rf,Random Forest Regressor,0.5131,0.6966,0.8223,0.2882,0.3290,3.8492,0.3620
et,Extra Trees Regressor,0.5286,0.7155,0.8368,0.2566,0.3287,4.1157,0.3290
xgboost,Extreme Gradient Boosting,0.5262,0.7403,0.8497,0.2338,0.3359,3.9605,0.4200
lr,Linear Regression,0.5446,0.7603,0.8607,0.2187,0.3511,2.8707,0.5920
ridge,Ridge Regression,0.5431,0.7618,0.8614,0.2175,0.3522,2.8654,0.0100
br,Bayesian Ridge,0.5437,0.7637,0.8623,0.2161,0.3560,2.6810,0.0220
huber,Huber Regressor,0.5263,0.7742,0.8664,0.2106,0.3522,2.8080,0.1680
knn,K Neighbors Regressor,0.5503,0.7845,0.8745,0.1903,0.3390,3.5448,0.0210


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4642,0.3830,0.6189,0.3917,0.3042,2.1609
1,0.5388,0.7603,0.8719,0.3483,0.3669,4.8889
2,0.4994,0.5569,0.7463,0.2955,0.3279,3.1422
3,0.5328,1.1096,1.0534,0.2340,0.3431,1.3352
4,0.4851,0.6010,0.7753,0.3552,0.3187,7.4189
5,0.5425,1.0763,1.0375,0.2054,0.3597,2.2628
6,0.4840,0.4895,0.6996,0.3884,0.3236,2.3619
7,0.4760,0.4063,0.6374,0.4065,0.2920,2.4445
8,0.5125,0.7564,0.8697,0.2627,0.3302,8.2305


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [24]:
# Stacking 

best_model = compare_models()

top_models = compare_models(n_select=3)

# Stack the top models
stacker = stack_models(estimator_list=top_models, meta_model=best_model)  
# 'best_model' is the model you want to use as the meta-model for stacking

# Evaluate the stacked model
evaluate_model(stacker)

Processing:   0%|          | 0/81 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5046,0.6659,0.8030,0.3218,0.3262,3.9078,0.1100
gbr,Gradient Boosting Regressor,0.5175,0.6945,0.8200,0.2934,0.3423,3.3300,0.1780
rf,Random Forest Regressor,0.5131,0.6966,0.8223,0.2882,0.3290,3.8492,0.4060
et,Extra Trees Regressor,0.5286,0.7155,0.8368,0.2566,0.3287,4.1157,0.3960
xgboost,Extreme Gradient Boosting,0.5262,0.7403,0.8497,0.2338,0.3359,3.9605,0.3890
lr,Linear Regression,0.5446,0.7603,0.8607,0.2187,0.3511,2.8707,0.0120
ridge,Ridge Regression,0.5431,0.7618,0.8614,0.2175,0.3522,2.8654,0.0100
br,Bayesian Ridge,0.5437,0.7637,0.8623,0.2161,0.3560,2.6810,0.0220
huber,Huber Regressor,0.5263,0.7742,0.8664,0.2106,0.3522,2.8080,0.1720
knn,K Neighbors Regressor,0.5503,0.7845,0.8745,0.1903,0.3390,3.5448,0.0210


Processing:   0%|          | 0/83 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4859,0.4152,0.6444,0.3406,0.3100,2.1263
1,0.5374,0.7209,0.8491,0.3821,0.3501,4.2866
2,0.5098,0.5654,0.7519,0.2847,0.3378,2.0475
3,0.5554,1.1469,1.0710,0.2082,0.3418,1.4937
4,0.5108,0.6798,0.8245,0.2707,0.3270,6.3288
5,0.5607,1.1403,1.0678,0.1582,0.3594,2.7376
6,0.4953,0.5320,0.7294,0.3353,0.3316,3.3915
7,0.5065,0.4867,0.6976,0.2890,0.3059,2.5824
8,0.5378,0.8192,0.9051,0.2015,0.3448,9.4157


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider at every split
    'max_depth': [None, 10, 20, 30],  # Maximum number of levels in the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Perform the grid search
grid_search.fit(train_df_X, train_df_Y["total_retard_mean"])

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_rf = RandomForestRegressor(random_state=42, **best_params)
best_rf.fit(train_df_X, train_df_Y["total_retard_mean"])

# Make predictions
predictions = best_rf.predict(test_df_X)


"""
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
"""


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [20]:
from sklearn.metrics import mean_squared_error, r2_score

# Get the best estimator from the grid search
best_rf = grid_search.best_estimator_

# Make predictions on the test data
predictions = best_rf.predict(test_df_X)

# Calculate and print evaluation metrics
mse = mean_squared_error(test_df_Y["total_retard_mean"], predictions)
r2 = r2_score(test_df_Y["total_retard_mean"], predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)




Mean Squared Error: 0.6462476944277021
R-squared: 0.32823671754812966


In [44]:

from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

# Sample data (replace this with your actual data)


# Define the hyperparameters grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 4, 5],
    'num_leaves':[10,20,30]
}

# Create LGBMRegressor model
lgbm = LGBMRegressor(random_state=42)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, n_jobs=-1, verbose = 2)

# Perform the grid search
grid_search.fit(train_df_X, train_df_Y["total_retard_mean"])

# Get the best parameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_lgbm = LGBMRegressor(random_state=42, **best_params)
best_lgbm.fit(train_df_X, train_df_Y["total_retard_mean"])

# Now you can use the best_lgbm model for predictions


Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000577 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1153
[LightGBM] [Info] Number of data points in the train set: 5942, number of used features: 124
[LightGBM] [Info] Start training from score -0.039694
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

LGBMRegressor(max_depth=5, n_estimators=200, num_leaves=30, random_state=42)

In [46]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Make predictions on the test data
predictions = best_lgbm.predict(test_df_X)

# Calculate evaluation metrics
mae = mean_absolute_error(test_df_Y["total_retard_mean"], predictions)
mse = mean_squared_error(test_df_Y["total_retard_mean"], predictions)
r2 = r2_score(test_df_Y["total_retard_mean"], predictions)

# Print the metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

Mean Absolute Error (MAE): 0.5005376936538753
Mean Squared Error (MSE): 0.6283951747667083
R-squared (R2): 0.3467941024501
